# Packages

In [1]:
import numpy as np
import pandas as pd
from astropy.table import Table
from scipy.io import readsav as read_idl

# Ancillary functions

In [2]:
# get flat array from list of arrays
def get_flat_data(data_list):
    """
    Takes a list of arrays and concatenates all arrays of the list,
    thus returns a single array.
    """
    data_array = []
    for x in data_list:
        # cast to array if float
        if len(x) == 0:
            continue
        if type(x) != np.ndarray:
            x = np.array([x])
        # concatenate to flat array
        data_array = np.concatenate((data_array, x))
        
    return data_array

# CO(2-1)/CO(1-0) calibration from Schinnerer & Leroy+24 (Tab. 1)
def get_R21_from_SFR(Sd_sfr):
    """
    Takes (resolved) star formation rate surface density and returns
    (resolved) CO(2-1)/CO(1-0) (R_21) line ratio.
    Can take multi-dim. arrays.
    
    Input:
    - Sd_sfr : star formation rate surface density [Msun/yr/kpc2]
    
    Output:
    - R_21   : CO(2-1)/CO(1-0) line ratio [dimensionless]
    """

    # set negatives to 0
    sfr = np.copy(Sd_sfr)
    sfr[sfr<0] = 0
    
    # apply scaling relation from table 1
    R_21 = 0.65 * (sfr/1.8e-2)**0.125


    # apply limits (table 1)
    R_21[R_21<0.35] = 0.35
    R_21[R_21>1] = 1
    
    return R_21


# define function to compute pixel area
def get_pixel_area(beam_size_as, dist_mpc, spb=2, pix_shape='gauss'):
    """
    INPUT:
    beam_size_as: beamsize (FWHM) in arcseconds
    dist_mpc: distance in mega parsec
    spb: samples per beam

    OUTPUT:
    A: pixel area in square parsec
    """
    # convert beam size to arcseconds
    as2pc = 1/3600 * np.pi/180 * dist_mpc * 1e6  # conversion from arcsecond to parsec
    beam_size_pc = beam_size_as * as2pc  # beamsize (FWHM) in parsec
    
    # compute area of pixel
    if pix_shape == 'gauss':
        # 2D Gaussian
        fwhm_to_sigma = 1. / (8 * np.log(2))**0.5
        A = 2 * np.pi * (beam_size_pc * fwhm_to_sigma)**2
    elif pix_shape == 'hex':
        # hexagon
        A = np.sqrt(3)/2 * (beam_size_pc/spb)**2
    elif pix_shape == 'circ':
        # circle
        A = np.pi * (beam_size_pc/spb/2)**2
    elif pix_shape == 'square':
        # square
        A = (beam_size_pc/spb)**2
    
    return A

# Setup
Produce compilation of HCN surveys paired with SFR (or IR) infrared measurements.
For resolved measurements, convert surface density units to luminosity units via beam-size apertures, where the area of a 2D Gaussian is given by $2\pi\sigma^2$, with $\sigma = 2\sqrt{2\,\ln 2}\,\mathrm{FWHM}$.

In [3]:
# make lists to store data
Refs = []
L_HCN = []
L_CO = []
L_TIR = []
M_dense = []
M_mol = []
SFR = []
Types = []
Sources = []
Refs_long = []
Dates = []

# directories
wd = '/Users/lneumann/Documents/'

# conversions
tir2sfr = 1.48e-10  # [Msun/yr] Murphy+11
# alpha_hcn = 10 # [Msun/pc2 / (K km s-1)] Gao+04
alpha_hcn = 15  # [M_Sun/pc^2/(K km/s)] (Leroy & Schinnerer 2024)
alpha_co = 4.35 # [Msun/pc2 / (K km s-1)] Bolatto+13
R_21 = 0.65 # Leroy+22, den Brok+21

# Literature data from Jimenez-Donaire+19

In [4]:
data_dir = wd + 'Data/MISC/dense_gas_literature/'

# complete literature compilation from EMPIRE paper
table = Table.read(data_dir + 'alldata_MJD.csv')

# remove Kepley+14 since it is loaded from another file
table = table[table['Ref']!='Kepley14']

# corrections to Braine+16 (according to MJD)
table['logLHCN'][table['Ref']=='Braine16'] -= 0.22
table['logLIR'][table['Ref']=='Braine16'] -= 1.3

l_hcn = 10**table['logLHCN']  # K.km/s.pc2
# l_co = 10**table['logLCO']  # K.km/s.pc2 (TBD, for now just a placeholder)
l_co = np.full_like(l_hcn, np.nan)  # K.km/s.pc2 (TBD, for now just a placeholder)
l_tir = 10**table['logLIR']  # Lsun
ref = list(table['Ref'])
types = list(table['Type'])

# compute sfr and mdense
sfr = l_tir * tir2sfr
m_dense = l_hcn * alpha_hcn
m_mol = l_co * alpha_co

# edit types
for i in range(len(types)):
    if types[i] == 'R':
        types[i] = 'parts of galaxies'
    elif types[i] == 'I':
        types[i] = 'clouds and clumps'
    else:
        types[i] = 'entire galaxies'
    # Chen+17 labelled as clouds, but is actually resolved galaxy (M51) data
    if ref[i] == 'Chen17':
        types[i] = 'parts of galaxies'

# edit refs
for i in range(len(ref)):
    if ref[i] == 'Braine16':
        ref[i] = 'Braine17' # Braine16 is actually from 2017 (2017A&A...597A..44B)

# add proper reference and publication date
ref_long = np.copy(ref)
date = np.copy(ref)

# Chin97
ref_long = ['1997A&A...317..548C' if x=='Chin97' else x for x in ref_long]
date = ['01/1997' if x=='Chin97' else x for x in date]

# Chin98
ref_long = ['1998A&A...330..901C' if x=='Chin98' else x for x in ref_long]
date = ['02/1998' if x=='Chin98' else x for x in date]

# Gao04
ref_long = ['2004ApJ...606..271G' if x=='Gao04' else x for x in ref_long]
date = ['05/2004' if x=='Gao04' else x for x in date]

# Brouillet05
ref_long = ['2005A&A...429..153B' if x=='Brouillet05' else x for x in ref_long]
date = ['01/2005' if x=='Brouillet05' else x for x in date]

# Gao07
ref_long = ['2007ApJ...660L..93G' if x=='Gao07' else x for x in ref_long]
date = ['05/2007' if x=='Gao07' else x for x in date]

# Gracia-Carpio08
ref_long = ['2008A&A...479..703G' if x=='Gracia-Carpio08' else x for x in ref_long]
date = ['03/2008' if x=='Gracia-Carpio08' else x for x in date]

# Krips08
ref_long = ['2008ApJ...677..262K' if x=='Krips08' else x for x in ref_long]
date = ['04/2008' if x=='Krips08' else x for x in date]

# Juneau09
ref_long = ['2009ApJ...707.1217J' if x=='Juneau09' else x for x in ref_long]
date = ['12/2009' if x=='Juneau09' else x for x in date]

# Wu10
ref_long = ['2010ApJS..188..313W' if x=='Wu10' else x for x in ref_long]
date = ['06/2010' if x=='Wu10' else x for x in date]

# Garcia-Burillo12
ref_long = ['2012A&A...539A...8G' if x=='Garcia-Burillo12' else x for x in ref_long]
date = ['03/2012' if x=='Garcia-Burillo12' else x for x in date]

# Crocker12
ref_long = ['2012MNRAS.421.1298C' if x=='Crocker12' else x for x in ref_long]
date = ['04/2012' if x=='Crocker12' else x for x in date]

# Buchbender13
ref_long = ['2013A&A...549A..17B' if x=='Buchbender13' else x for x in ref_long]
date = ['01/2013' if x=='Buchbender13' else x for x in date]

# Usero15
ref_long = ['2015AJ....150..115U' if x=='Usero15' else x for x in ref_long]
date = ['10/2015' if x=='Usero15' else x for x in date]

# Privon15
ref_long = ['2015ApJ...814...39P' if x=='Privon15' else x for x in ref_long]
date = ['11/2015' if x=='Privon15' else x for x in date]

# Bigiel15
ref_long = ['2015ApJ...815..103B' if x=='Bigiel15' else x for x in ref_long]
date = ['12/2015' if x=='Bigiel15' else x for x in date]

# Stephens16
ref_long = ['2016ApJ...824...29S' if x=='Stephens16' else x for x in ref_long]
date = ['06/2016' if x=='Stephens16' else x for x in date]

# Braine17
ref_long = ['2017A&A...597A..44B' if x=='Braine17' else x for x in ref_long]
date = ['01/2017' if x=='Braine17' else x for x in date]

# Chen17
ref_long = ['2017ApJ...836..101C' if x=='Chen17' else x for x in ref_long]
date = ['02/2017' if x=='Chen17' else x for x in date]

# Gallagher18
ref_long = ['2018ApJ...858...90G' if x=='Gallagher18' else x for x in ref_long]
date = ['05/2018' if x=='Gallagher18' else x for x in date]

# append to list
Refs = Refs + ref
L_HCN = np.concatenate((L_HCN, l_hcn))
M_dense = np.concatenate((M_dense, m_dense))
L_CO = np.concatenate((L_CO, l_co))
M_mol = np.concatenate((M_mol, m_mol))
L_TIR = np.concatenate((L_TIR, l_tir))
SFR = np.concatenate((SFR, sfr))
Types = Types + types
Sources = Sources + ['NaN']*len(sfr)
Refs_long = Refs_long + ref_long
Dates = Dates + date

# Kepley+14 (M82)

In [5]:
table_M82 = pd.read_csv(data_dir + 'kepley_m82.csv', header=[0])
table_M82 = table_M82.drop(0)
l_hcn = np.array(table_M82['L_HCN']).astype('float64')  # K.km/s.pc2
l_co = np.array(table_M82['L_CO']).astype('float64')  # K.km/s.pc2 
l_tir = np.array(table_M82['L_IR']).astype('float64')  # Lsun

# compute sfr and mdense
sfr = l_tir * tir2sfr
m_dense = l_hcn * alpha_hcn
m_mol = l_co * alpha_co

# append to list
Refs = Refs + ['Kepley14']*len(sfr)
L_HCN = np.concatenate((L_HCN, l_hcn))
M_dense = np.concatenate((M_dense, m_dense))
L_CO = np.concatenate((L_CO, l_co))
M_mol = np.concatenate((M_mol, m_mol))
L_TIR = np.concatenate((L_TIR, l_tir))
SFR = np.concatenate((SFR, sfr))
Types = Types + ['parts of galaxies']*len(sfr)
Sources = Sources + ['ngc3034']*len(sfr)
Refs_long = Refs_long + ['2014ApJ...780L..13K']*len(sfr)
Dates = Dates + ['01/2014']*len(sfr)

# MW CMZ

In [6]:
table_MW = Table.read(data_dir + 'MWCMZ_MJD.csv')
l_hcn = 10**table_MW['logLHCN']  # K.km/s.pc2
# l_co = 10**table['logLCO']  # K.km/s.pc2 (TBD, for now just a placeholder)
l_co = np.full_like(l_hcn, np.nan)  # K.km/s.pc2 (TBD, for now just a placeholder)
l_tir = 10**table_MW['logLIR']  # Lsun

# compute sfr and mdense
sfr = l_tir * tir2sfr
m_dense = l_hcn * alpha_hcn
m_mol = l_co * alpha_co

# append to list
Refs = Refs + ['Henshaw23']*len(sfr)
L_HCN = np.concatenate((L_HCN, l_hcn))
M_dense = np.concatenate((M_dense, m_dense))
L_CO = np.concatenate((L_CO, l_co))
M_mol = np.concatenate((M_mol, m_mol))
L_TIR = np.concatenate((L_TIR, l_tir))
SFR = np.concatenate((SFR, sfr))
Types = Types + ['clouds and clumps']*len(sfr)
Sources = Sources + ['MW CMZ']*len(sfr)
Refs_long = Refs_long + ['2023ASPC..534...83H']*len(sfr)
Dates = Dates + ['07/2023']*len(sfr)

# Galactic clouds

In [7]:
# Lada+12
table_cl = Table.read(data_dir + 'clouds_lada.csv')
m_dense = table_cl['Mdense']  # Msun
sfr = table_cl['SFR']  # Msun/yr

# compute HCN luminosity and IR luminosity
l_tir = sfr / tir2sfr
l_hcn = m_dense / alpha_hcn
l_co = np.full_like(l_hcn, np.nan)
m_mol = l_co * alpha_co

# append to list
Refs = Refs + ['Lada12']*len(sfr)
L_HCN = np.concatenate((L_HCN, l_hcn))
M_dense = np.concatenate((M_dense, m_dense))
L_CO = np.concatenate((L_CO, l_co))
M_mol = np.concatenate((M_mol, m_mol))
L_TIR = np.concatenate((L_TIR, l_tir))
SFR = np.concatenate((SFR, sfr))
Types = Types + ['clouds and clumps']*len(sfr)
Sources = Sources + ['MW clouds']*len(sfr)
Refs_long = Refs_long + ['2012ApJ...745..190L']*len(sfr)
Dates = Dates + ['02/2012']*len(sfr)

# Evans+14
table_ce = Table.read(data_dir + 'clouds_evans.csv')
m_dense = table_ce['Mdense']  # Msun
m_mol = table_ce['Mcloud']  # Msun
sfr = table_ce['SFR']  # Msun/yr

# compute HCN luminosity and IR luminosity
l_tir = sfr / tir2sfr
l_hcn = m_dense / alpha_hcn
l_co = m_mol / alpha_co

# append to list
Refs = Refs + ['Evans14']*len(sfr)
L_HCN = np.concatenate((L_HCN, l_hcn))
M_dense = np.concatenate((M_dense, m_dense))
L_CO = np.concatenate((L_CO, l_co))
M_mol = np.concatenate((M_mol, m_mol))
L_TIR = np.concatenate((L_TIR, l_tir))
SFR = np.concatenate((SFR, sfr))
Types = Types + ['clouds and clumps']*len(sfr)
Sources = Sources + ['MW clouds']*len(sfr)
Refs_long = Refs_long + ['2014ApJ...782..114E']*len(sfr)
Dates = Dates + ['02/2014']*len(sfr)

# ALMOND (Neumann+23)

In [8]:
# galaxies
glxys = ['ngc0628','ngc1097','ngc1365','ngc1385','ngc1511','ngc1546','ngc1566','ngc1672','ngc1792','ngc2566',
         'ngc2903','ngc2997','ngc3059','ngc3521','ngc3621','ngc4303','ngc4321','ngc4535','ngc4536','ngc4569',
         'ngc4826','ngc5248','ngc5643','ngc6300','ngc7496']
# resolutions
res_list = ['18.6as', '19.4as', '20.6as', '19.9as', '17.6as', '18.9as', '19.7as', '18.2as', '18.7as', '18.5as',
            '18.3as', '20.4as', '16.7as', '21.1as', '18.9as', '20.2as', '19.6as', '22.8as', '21.5as', '19.2as',
            '18.7as', '19.9as', '18.0as', '17.7as', '17.9as']

cloudres_list = ['1.1as', '1.6as', '1.3as', '1.2as', '1.4as', '1.2as', '1.2as', '1.9as', '1.9as', '1.2as',
                 '1.4as', '1.7as', '1.2as', '1.3as', '1.8as', '1.8as', '1.6as', '1.5as', '1.4as', '1.6as', 
                 '1.2as', '1.2as', '1.2as', '1.0as', '1.6as']

# create lists to store data
hcn_list, hcn_err_list = [],[]
Sd_dense_list, Sd_dense_err_list = [],[]
Sd_mol_list, Sd_mol_err_list = [],[]
m_dense_list, m_dense_err_list = [],[]
m_mol_list, m_mol_err_list = [],[]
Sd_sfr_list, Sd_sfr_err_list = [],[]
sfr_list, sfr_err_list = [],[]
L_tir_list, L_tir_err_list = [],[]
dist_list = []
incl_list = []
glxy_list = []

# loop over galaxies
for glxy, res, cloudres in zip(glxys, res_list, cloudres_list):

    ###################################################################
    # LOAD DATA
    ###################################################################
    
    # PYSTRUCTURE data
    pystruct_file = wd + 'Data/ALMOND/pystruct/' + glxy + '_data_struct_' + res +'_spb2.npy'
    # pystruct_file = wd + 'Data/ALMOND/pystruct/' + glxy + '_data_struct_' + res +'_spb1.npy'
    pystruct = np.load(pystruct_file, allow_pickle = True).item()
    
    # get data from structure
    dist_Mpc = pystruct['dist_mpc']  # distance to the galaxy in Mpc
    incl = pystruct['incl_deg']  # inclination of the galaxy in degree
    beam_size_as = pystruct['beam_as']  # beam size in arcseconds
    hcn = pystruct['INT_VAL_HCN10']
    hcn_err = pystruct['INT_UC_HCN10']
    co21 = pystruct['INT_VAL_CO21']
    co21_err = pystruct['INT_UC_CO21']
    Sd_sfr = pystruct['INT_VAL_SFR_Z0MGS'] * 1e-6  # [Msun/yr/pc2]
    Sd_sfr_err = pystruct['INT_UC_SFR_Z0MGS'] * 1e-6  # [Msun/yr/pc2]

    # put zeros to nan
    mask = (hcn == 0) | (hcn_err == 0) | (Sd_sfr == 0) | (Sd_sfr_err == 0)
    hcn[mask] = np.nan
    hcn_err[mask] = np.nan
    co21[mask] = np.nan
    co21_err[mask] = np.nan
    Sd_sfr[mask] = np.nan
    Sd_sfr_err[mask] = np.nan

    # convert CO(2-1) into CO(1-0) via R_21 calibration from SL+24
    # R_21 = get_R21_from_SFR(Sd_sfr)
    R_21 = pystruct['INT_VAL_R21_SL24']
    co10 = co21 / R_21
    co10_err = co21_err / R_21

    # compute molecular gas surface density
    Sd_mol = alpha_co * co10  # [Msun/pc2]
    Sd_mol_err = alpha_co * co10_err # [Msun/pc2]
    
    # compute dense gas surface density
    Sd_dense = alpha_hcn * hcn  # [Msun/pc2]
    Sd_dense_err = alpha_hcn * hcn_err # [Msun/pc2]
    
    # compute dense gas mass and sfr
    # A_pix_pc2 = get_pixel_area(beam_size_as, dist_Mpc, spb=2, pix_shape='hex')
    A_beam_pc2 = get_pixel_area(beam_size_as, dist_Mpc, spb=1, pix_shape='gauss')
    A_pc2 = A_beam_pc2
    m_mol = Sd_mol * A_pc2  # [Msun]
    m_mol_err = Sd_mol_err * A_pc2  # [Msun]
    m_dense = Sd_dense * A_pc2  # [Msun]
    m_dense_err = Sd_dense_err * A_pc2  # [Msun]
    sfr = Sd_sfr * A_pc2  # [Msun/yr]
    sfr_err = Sd_sfr_err * A_pc2  # [Msun/yr]
    
    # compute total infrared luminosity
    L_tir = sfr / tir2sfr
    L_tir_err = sfr_err / tir2sfr
    
    # append to lists
    hcn_list.append(hcn)
    hcn_err_list.append(hcn_err)
    Sd_mol_list.append(Sd_mol)
    Sd_mol_err_list.append(Sd_mol_err)
    Sd_dense_list.append(Sd_dense)
    Sd_dense_err_list.append(Sd_dense_err)
    m_dense_list.append(m_dense)
    m_dense_err_list.append(m_dense_err)
    m_mol_list.append(m_mol)
    m_mol_err_list.append(m_mol_err)
    
    Sd_sfr_list.append(Sd_sfr)
    Sd_sfr_err_list.append(Sd_sfr_err)
    sfr_list.append(sfr)
    sfr_err_list.append(sfr_err)
    L_tir_list.append(L_tir)
    L_tir_err_list.append(L_tir_err)
    
    dist_list.append(dist_Mpc)
    incl_list.append(incl)
           
    glxy_list += [glxy]*len(hcn)

# convert lists of data for each galaxy to 1D arrays
hcn_almond = get_flat_data(hcn_list)
hcn_err_almond = get_flat_data(hcn_err_list)
Sd_dense_almond = get_flat_data(Sd_dense_list)
Sd_dense_err_almond = get_flat_data(Sd_dense_err_list)
m_dense_almond = get_flat_data(m_dense_list)
m_dense_err_almond = get_flat_data(m_dense_err_list)
m_mol_almond = get_flat_data(m_mol_list)
m_mol_err_almond = get_flat_data(m_mol_err_list)

Sd_sfr_almond = get_flat_data(Sd_sfr_list)
Sd_sfr_err_almond = get_flat_data(Sd_sfr_err_list)
sfr_almond = get_flat_data(sfr_list)
sfr_err_almond = get_flat_data(sfr_err_list)
L_tir_almond = get_flat_data(L_tir_list)
L_tir_err_almond = get_flat_data(L_tir_err_list)

dist_almond = np.array(dist_list)
incl_almond = np.array(incl_list)


# append to list
det = ((hcn_almond / hcn_err_almond) > 3) & (sfr_almond/sfr_err_almond > 3)
Refs = Refs + ['Neumann23']*len(sfr_almond[det])
L_HCN = np.concatenate((L_HCN, m_dense_almond[det]/alpha_hcn))
M_dense = np.concatenate((M_dense, m_dense_almond[det]))
L_CO = np.concatenate((L_CO, m_mol_almond[det]/alpha_co))
M_mol = np.concatenate((M_mol, m_mol_almond[det]))
L_TIR = np.concatenate((L_TIR, L_tir_almond[det]))
SFR = np.concatenate((SFR, sfr_almond[det]))
Types = Types + ['parts of galaxies']*len(sfr_almond[det])
Sources = Sources + [glxy for glxy, mask in zip(glxy_list, det) if mask]
Refs_long = Refs_long + ['2023MNRAS.521.3348N']*len(sfr_almond[det])
Dates = Dates + ['05/2023']*len(sfr_almond[det])

# NGC 4321 (Neumann+24)

In [9]:
# pystructure file
pystruct_file = wd + 'Data/NGC4321/pystruct/' + 'ngc4321_data_struct_3.6as_spb2.npy'
pystruct = np.load(pystruct_file, allow_pickle = True).item()

# get data from structure
dist_Mpc = pystruct['dist_mpc']  # distance to the galaxy in Mpc
incl = pystruct['incl_deg']  # inclination of the galaxy in degree
beam_size_as = pystruct['beam_as']  # beam size in arcseconds
hcn = pystruct['INT_VAL_HCN10']
hcn_err = pystruct['INT_UC_HCN10']
co21 = pystruct['INT_VAL_CO21']
co21_err = pystruct['INT_UC_CO21']
Sd_sfr = pystruct['INT_VAL_SFR_HACORR'] * 1e-6 # [Msun/yr\pc2]
Sd_sfr_err = pystruct['INT_UC_SFR_HACORR'] * 1e-6

# put zeros to nan
mask = (hcn == 0) | (hcn_err == 0) | (Sd_sfr == 0) | (Sd_sfr_err == 0)
hcn[mask] = np.nan
hcn_err[mask] = np.nan
co21[mask] = np.nan
co21_err[mask] = np.nan
Sd_sfr[mask] = np.nan
Sd_sfr_err[mask] = np.nan

# convert CO(2-1) into CO(1-0) via R_21 calibration from Leroy+22
R_21 = get_R21_from_SFR(Sd_sfr)
co10 = co21 / R_21
co10_err = co21_err / R_21

# compute dense gas surface density
Sd_dense = alpha_hcn * hcn  # [Msun/pc2]
Sd_dense_err = alpha_hcn * hcn_err # [Msun/pc2]

# compute dense gas surface density
Sd_mol = alpha_co * co10  # [Msun/pc2]
Sd_mol_err = alpha_co * co10_err # [Msun/pc2]

# compute dense gas mass and sfr
# A_pix_pc2 = get_pixel_area(beam_size_as, dist_Mpc, spb=2, pix_shape='hex')
A_beam_pc2 = get_pixel_area(beam_size_as, dist_Mpc, spb=1, pix_shape='gauss')
A_pc2 = A_beam_pc2
m_mol = Sd_mol * A_pc2  # [Msun]
m_mol_err = Sd_mol_err * A_pc2  # [Msun]
m_dense = Sd_dense * A_pc2  # [Msun]
m_dense_err = Sd_dense_err * A_pc2  # [Msun]
sfr = Sd_sfr * A_pc2  # [Msun/yr]
sfr_err = Sd_sfr_err * A_pc2  # [Msun/yr]

# compute total infrared luminosity
tir2sfr = 1.48e-10  # [Msun/yr] Murphy+11
L_tir = sfr / tir2sfr
L_tir_err = sfr_err / tir2sfr

# append to list
det = ((hcn / hcn_err) > 3) & (sfr/sfr_err > 3)
Refs = Refs + ['Neumann24']*len(sfr[det])
L_HCN = np.concatenate((L_HCN, m_dense[det]/alpha_hcn))
M_dense = np.concatenate((M_dense, m_dense[det]))
L_CO = np.concatenate((L_CO, m_mol[det]/alpha_co))
M_mol = np.concatenate((M_mol, m_mol[det]))
L_TIR = np.concatenate((L_TIR, L_tir[det]))
SFR = np.concatenate((SFR, sfr[det]))
Types = Types + ['parts of galaxies']*len(sfr[det])
Sources = Sources + ['ngc4321']*len(sfr[det])
Refs_long = Refs_long + ['2024A&A...691A.121N']*len(sfr[det])
Dates = Dates + ['11/2024']*len(sfr[det])

# EMPIRE (Jimenez-Donaire+19)

In [10]:
empire_galaxies = ['ngc0628', 'ngc2903', 'ngc3184', 'ngc3627', 'ngc4254', 'ngc4321', 'ngc5055', 'ngc5194', 'ngc6946']
empire_res_list = ['33.3as', '33.3as', '33.3as', '33.3as', '33.3as', '33.3as', '33.3as', '33.3as', '33.3as']

# create lists to store data
hcn_list, hcn_err_list = [],[]
Sd_dense_list, Sd_dense_err_list = [],[]
Sd_mol_list, Sd_mol_err_list = [],[]
m_dense_list, m_dense_err_list = [],[]
m_mol_list, m_mol_err_list = [],[]
Sd_sfr_list, Sd_sfr_err_list = [],[]
sfr_list, sfr_err_list = [],[]
L_tir_list, L_tir_err_list = [],[]
dist_list = []
incl_list = []
glxy_list = []

for glxy, res in zip(empire_galaxies, empire_res_list):

    ###################################################################
    # LOAD DATA
    ###################################################################
    
    # PYSTRUCTURE data
    pystruct_file = wd + 'Data/EMPIRE/pystruct/' + glxy + '_data_struct_' + res +'_spb2.npy'
    pystruct = np.load(pystruct_file, allow_pickle = True).item()
    
    # get data from structure
    dist_Mpc = pystruct['dist_mpc']  # distance to the galaxy in Mpc
    incl = pystruct['incl_deg']  # inclination of the galaxy in degree
    beam_size_as = pystruct['beam_as']  # beam size in arcseconds
    hcn = pystruct['INT_VAL_HCN10']
    hcn_err = pystruct['INT_UC_HCN10']
    co10 = pystruct['INT_VAL_CO10']
    co10_err = pystruct['INT_UC_CO10']
    Sd_sfr = pystruct['INT_VAL_SFR_Z0MGS'] * 1e-6  # [Msun/yr/pc2]
    Sd_sfr_err = pystruct['INT_UC_SFR_Z0MGS'] * 1e-6  # [Msun/yr/pc2]
    
    # put zeros to nan
    mask = (hcn == 0) | (Sd_sfr == 0)
    hcn[mask] = np.nan
    hcn_err[mask] = np.nan
    co10[mask] = np.nan
    co10_err[mask] = np.nan
    Sd_sfr[mask] = np.nan
    Sd_sfr_err[mask] = np.nan
    
    # compute molecular gas surface density
    Sd_mol = alpha_co * co10  # [Msun/pc2]
    Sd_mol_err = alpha_co * co10_err # [Msun/pc2]
    
    # compute dense gas surface density
    Sd_dense = alpha_hcn * hcn  # [Msun/pc2]
    Sd_dense_err = alpha_hcn * hcn_err # [Msun/pc2]
    
    # compute dense gas mass and sfr
    # A_pix_pc2 = get_pixel_area(beam_size_as, dist_Mpc, spb=2, pix_shape='hex')
    A_beam_pc2 = get_pixel_area(beam_size_as, dist_Mpc, spb=1, pix_shape='gauss')
    A_pc2 = A_beam_pc2
    m_mol = Sd_mol * A_pc2  # [Msun]
    m_mol_err = Sd_mol_err * A_pc2  # [Msun]
    m_dense = Sd_dense * A_pc2  # [Msun]
    m_dense_err = Sd_dense_err * A_pc2  # [Msun]
    sfr = Sd_sfr * A_pc2 # [Msun/yr]
    sfr_err = Sd_sfr_err * A_pc2 # [Msun/yr]
    
    # compute total infrared luminosity
    L_tir = sfr / tir2sfr
    L_tir_err = sfr_err / tir2sfr
    
    # append to lists
    hcn_list.append(hcn)
    hcn_err_list.append(hcn_err)
    Sd_mol_list.append(Sd_mol)
    Sd_mol_err_list.append(Sd_mol_err)
    Sd_dense_list.append(Sd_dense)
    Sd_dense_err_list.append(Sd_dense_err)
    m_dense_list.append(m_dense)
    m_dense_err_list.append(m_dense_err)
    m_mol_list.append(m_mol)
    m_mol_err_list.append(m_mol_err)
    
    Sd_sfr_list.append(Sd_sfr)
    Sd_sfr_err_list.append(Sd_sfr_err)
    sfr_list.append(sfr)
    sfr_err_list.append(sfr_err)
    L_tir_list.append(L_tir)
    L_tir_err_list.append(L_tir_err)
    
    dist_list.append(dist_Mpc)
    incl_list.append(incl)
    
    glxy_list += [glxy]*len(hcn)
    
# convert lists of data for each galaxy to 1D arrays
hcn_empire = get_flat_data(hcn_list)
hcn_err_empire = get_flat_data(hcn_err_list)
Sd_dense_empire = get_flat_data(Sd_dense_list)
Sd_dense_err_empire = get_flat_data(Sd_dense_err_list)
m_dense_empire = get_flat_data(m_dense_list)
m_dense_err_empire = get_flat_data(m_dense_err_list)
m_mol_empire = get_flat_data(m_mol_list)
m_mol_err_empire = get_flat_data(m_mol_err_list)

Sd_sfr_empire = get_flat_data(Sd_sfr_list)
Sd_sfr_err_empire = get_flat_data(Sd_sfr_err_list)
sfr_empire = get_flat_data(sfr_list)
sfr_err_empire = get_flat_data(sfr_err_list)
L_tir_empire = get_flat_data(L_tir_list)
L_tir_err_empire = get_flat_data(L_tir_err_list)

dist_empire = np.array(dist_list)
incl_empire = np.array(incl_list)

# append to list
det = ((hcn_empire / hcn_err_empire) > 3) & (sfr_empire/sfr_err_empire > 3)
Refs = Refs + ['Jiménez-Donaire19']*len(sfr_empire[det])
L_HCN = np.concatenate((L_HCN, m_dense_empire[det]/alpha_hcn))
M_dense = np.concatenate((M_dense, m_dense_empire[det]))
L_CO = np.concatenate((L_CO, m_mol_empire[det]/alpha_co))
M_mol = np.concatenate((M_mol, m_mol_empire[det]))
L_TIR = np.concatenate((L_TIR, L_tir_empire[det]))
SFR = np.concatenate((SFR, sfr_empire[det]))
Types = Types + ['parts of galaxies']*len(sfr_empire[det])
Sources = Sources + [glxy for glxy, mask in zip(glxy_list, det) if mask]
Refs_long = Refs_long + ['2019ApJ...880..127J']*len(sfr_empire[det])
Dates = Dates + ['08/2019']*len(sfr_empire[det])

# M51 (Querejeta+19)

In [11]:
table = pd.read_csv(data_dir + 'm51_tablea2.csv', header=None) # all data
L_tir = table[5] * 1e6  # Lsun
L_tir_err = table[6] * 1e6
Sd_sfr = table[11] * 1e-6  # [Msun/yr/pc2]
Sd_sfr_err = table[12] * 1e-6  # [Msun/yr/pc2]
hcn = table[13]  # K.km/s
hcn_err = table[14]
co = table[15]  # K.km/s
co_err = table[16]

# compute SFR and Mdense
beam_size_pc = 100
fwhm_to_sigma = 1. / (8 * np.log(2))**0.5
A_beam_pc2 = 2 * np.pi * (beam_size_pc * fwhm_to_sigma)**2
# A_pix_pc2 = (beam_size_pc/2)**2
A_pc2 = A_beam_pc2
m_dense = hcn * alpha_hcn * A_pc2 # [Msun]
m_dense_err = hcn_err * alpha_hcn * A_pc2 # [Msun]
m_mol = co * alpha_co * A_pc2 # [Msun]
m_mol_err = co_err * alpha_co * A_pc2 # [Msun]
sfr = Sd_sfr * A_pc2  # [Msun/yr]
sfr_err = Sd_sfr_err * A_pc2  # [Msun/yr]
# sfr = L_tir * tir2sfr # [Msun/yr]
# sfr_err = L_tir_err * tir2sfr # [Msun/yr]

# append to list
det = ((hcn / hcn_err) > 3) & (sfr/sfr_err > 3)
Refs = Refs + ['Querejeta19']*len(sfr[det])
L_HCN = np.concatenate((L_HCN, m_dense[det]/alpha_hcn))
M_dense = np.concatenate((M_dense, m_dense[det]))
L_CO = np.concatenate((L_CO, m_mol[det]/alpha_co))
M_mol = np.concatenate((M_mol, m_mol[det]))
L_TIR = np.concatenate((L_TIR, L_tir[det]))
SFR = np.concatenate((SFR, sfr[det]))
Types = Types + ['parts of galaxies']*len(sfr[det])
Sources = Sources + ['ngc5194']*len(sfr[det])
Refs_long = Refs_long + ['2019A&A...625A..19Q']*len(sfr[det])
Dates = Dates + ['05/2019']*len(sfr[det])

# NGC 1068 (Sánchez-García+22)

In [12]:
# from their figure 8
table = pd.read_table(data_dir + 'NGC1068_hcn_fig8_MJ.txt', sep='  ', engine='python')
Sd_dense = 10**table['Sgas56'] # Msun/pc2
Sd_mol = np.full_like(Sd_dense, np.nan) # TBD (placeholder)
Sd_sfr = 10**table['Ssfr56'] * 1e-6  # [Msun/yr/pc2]
spb = 2  # samples per beam

# compute SFR and Mdense from surface densities
beam_size_pc = 56
fwhm_to_sigma = 1. / (8 * np.log(2))**0.5
A_beam_pc2 = 2 * np.pi * (beam_size_pc * fwhm_to_sigma)**2
# A_pix_pc2 = (beam_size_pc/2)**2
A_pc2 = A_beam_pc2
m_dense = Sd_dense * A_pc2  # [Msun]
m_mol = Sd_mol * A_pc2  # [Msun]
sfr = Sd_sfr * A_pc2  # [Msun/yr]

# compute IR and HCN luminosities
hcn = m_dense / alpha_hcn
co = m_mol / alpha_co
L_tir = sfr / tir2sfr

# append to list
Refs = Refs + ['Sánchez-García22']*len(sfr)
L_HCN = np.concatenate((L_HCN, hcn))
M_dense = np.concatenate((M_dense, m_dense))
L_CO = np.concatenate((L_CO, co))
M_mol = np.concatenate((M_mol, m_mol))
L_TIR = np.concatenate((L_TIR, L_tir))
SFR = np.concatenate((SFR, sfr))
Types = Types + ['parts of galaxies']*len(sfr)
Sources = Sources + ['ngc1068']*len(sfr)
Refs_long = Refs_long + ['2022A&A...660A..83S']*len(sfr)
Dates = Dates + ['04/2022']*len(sfr)

# NGC 3627 (Beslic+21)

In [13]:
# load table
table = read_idl(data_dir + 'ngc3627_database_nov2020.idl', python_dict=True)

# distance and resolution
beam_size_as = 1.85  # [arcsec]
dist_Mpc = 11.3  # Mpc (Beslic+21)

# load data from table
hcn = table['this_data']['INT_VAL_HCN10']
hcn_err = table['this_data']['INT_UC_HCN10']
co21 = table['this_data']['INT_VAL_CO21']
co21_err = table['this_data']['INT_UC_CO21']
Sd_sfr = table['this_data']['int_val_sfr'] * 1e-6  # note: SFR uncertainties not there

# mask nan values
mask_nan = (hcn != 0) & (~np.isnan(Sd_sfr))
hcn = hcn[mask_nan]
hcn_err = hcn_err[mask_nan]
co21 = co21[mask_nan]
co21_err = co21_err[mask_nan]
Sd_sfr = Sd_sfr[mask_nan]

# convert CO(2-1) into CO(1-0) via R_21 calibration from Schinnerer & Leroy 24
R_21 = get_R21_from_SFR(Sd_sfr)
co = co21 / R_21
co_err = co21_err / R_21

# compute dense gas surface density
Sd_dense = alpha_hcn * hcn  # [Msun/pc2]
Sd_dense_err = alpha_hcn * hcn_err  # [Msun/pc2]

# compute dense gas surface density
Sd_mol = alpha_co * co # [Msun/pc2]
Sd_mol_err = alpha_co * co_err # [Msun/pc2]

# compute dense gas mass and sfr
# A_pix_pc2 = get_pixel_area(beam_size_as, dist_Mpc, spb=2, pix_shape='hex')
A_beam_pc2 = get_pixel_area(beam_size_as, dist_Mpc, spb=1, pix_shape='gauss')
A_pc2 = A_beam_pc2
m_mol = Sd_mol * A_pc2  # [Msun]
m_mol_err = Sd_mol_err * A_pc2  # [Msun]
m_dense = Sd_dense * A_pc2  # [Msun]
m_dense_err = Sd_dense_err * A_pc2  # [Msun]
sfr = Sd_sfr * A_pc2  # [Msun/yr]

# compute total infrared luminosity
tir2sfr = 1.48e-10  # [Msun/yr] Murphy+11
L_tir = sfr / tir2sfr

# append to list
det = (hcn / hcn_err) > 3
Refs = Refs + ['Bešlić21']*len(sfr[det])
L_HCN = np.concatenate((L_HCN, m_dense[det]/alpha_hcn))
M_dense = np.concatenate((M_dense, m_dense[det]))
L_CO = np.concatenate((L_CO, m_mol[det]/alpha_co))
M_mol = np.concatenate((M_mol, m_mol[det]))
L_TIR = np.concatenate((L_TIR, L_tir[det]))
SFR = np.concatenate((SFR, sfr[det]))
Types = Types + ['parts of galaxies']*len(sfr[det])
Sources = Sources + ['ngc3627']*len(sfr[det])
Refs_long = Refs_long + ['2021MNRAS.506..963B']*len(sfr[det])
Dates = Dates + ['09/2021']*len(sfr[det])

# NGC 6946 (Eibensteiner+22)

In [14]:
# pystructure file
pystruct_file = data_dir + 'ngc6946_data_struct_4as_2021_10_18_for-corr.npy'
pystruct = np.load(pystruct_file, allow_pickle = True).item()

# get data from structure
dist_Mpc = pystruct['dist_mpc']  # distance to the galaxy in Mpc
incl = pystruct['incl_deg']  # inclination of the galaxy in degree
beam_size_as = pystruct['beam_as']  # beam size in arcseconds
hcn = pystruct['INT_VAL_HCN10']  # [K.km/s]
hcn_err = pystruct['INT_UC_HCN10']
co = pystruct['INT_VAL_CO10']  # [K.km/s]
co_err = pystruct['INT_UC_CO10']  
Sd_sfr = pystruct['INT_VAL_SFR_33GHZ_KPC'] * 1e-6  # [Msun/yr/pc2]

# put zeros to nan
mask = hcn == 0
hcn[mask] = np.nan
hcn_err[mask] = np.nan
co[mask] = np.nan
co_err[mask] = np.nan
Sd_sfr[mask] = np.nan

# compute dense gas surface density
Sd_dense = alpha_hcn * hcn  # [Msun/pc2]
Sd_dense_err = alpha_hcn * hcn_err # [Msun/pc2]

# compute dense gas surface density
Sd_mol = alpha_co * co  # [Msun/pc2]
Sd_mol_err = alpha_co * co_err # [Msun/pc2]

# compute dense gas mass and sfr
# A_pix_pc2 = get_pixel_area(beam_size_as, dist_Mpc, spb=2, pix_shape='hex')
A_beam_pc2 = get_pixel_area(beam_size_as, dist_Mpc, spb=1, pix_shape='gauss')
A_pc2 = A_beam_pc2
m_mol = Sd_mol * A_pc2  # [Msun]
m_mol_err = Sd_mol_err * A_pc2  # [Msun]
m_dense = Sd_dense * A_pc2  # [Msun]
m_dense_err = Sd_dense_err * A_pc2  # [Msun]
sfr = Sd_sfr * A_pc2  # [Msun/yr]

# compute total infrared luminosity
tir2sfr = 1.48e-10  # [Msun/yr] Murphy+11
L_tir = sfr / tir2sfr
# L_tir_err = sfr_err / tir2sfr

# append to list
det = (hcn / hcn_err) > 3
Refs = Refs + ['Eibensteiner22']*len(sfr[det])
L_HCN = np.concatenate((L_HCN, m_dense[det]/alpha_hcn))
M_dense = np.concatenate((M_dense, m_dense[det]))
L_CO = np.concatenate((L_CO, m_mol[det]/alpha_co))
M_mol = np.concatenate((M_mol, m_mol[det]))
L_TIR = np.concatenate((L_TIR, L_tir[det]))
SFR = np.concatenate((SFR, sfr[det]))
Types = Types + ['parts of galaxies']*len(sfr[det])
Sources = Sources + ['ngc6946']*len(sfr[det])
Refs_long = Refs_long + ['2022A&A...659A.173E']*len(sfr[det])
Dates = Dates + ['03/2022']*len(sfr[det])

# DSFGs (Rybak+22)

In [15]:
# data from their table 3
l_hcn = np.array([8e10])
l_co = np.array([np.nan])

# data from their table 4
l_hcn_fir = np.array([9.8e-4])

# compute ir, sfr and mdense
l_fir = l_hcn/l_hcn_fir
sfr = l_fir * tir2sfr
m_dense = l_hcn * alpha_hcn
m_mol = l_co * alpha_co

# append to list
Refs = Refs + ['Rybak22']*len(sfr)
L_HCN = np.concatenate((L_HCN, l_hcn))
M_dense = np.concatenate((M_dense, m_dense))
L_CO = np.concatenate((L_CO, l_co))
M_mol = np.concatenate((M_mol, m_mol))
L_TIR = np.concatenate((L_TIR, l_fir))
SFR = np.concatenate((SFR, sfr))
Types = Types + ['entire galaxies']*len(sfr)
Sources = Sources + ['J1202']
Refs_long = Refs_long + ['2022A&A...667A..70R']*len(sfr)
Dates = Dates + ['11/2022']*len(sfr)

# M51 (Stuber+23) 

In [16]:
# pystructure file
pystruct_file = wd + 'Data/NGC5194/pystruct/' + 'ngc5194_data_struct_3as_2024_08_21_with_alphaCO.npy'
pystruct = np.load(pystruct_file, allow_pickle = True).item()

# get data from structure
dist_Mpc = pystruct['dist_mpc']  # distance to the galaxy in Mpc
incl = pystruct['incl_deg']  # inclination of the galaxy in degree
beam_size_as = pystruct['beam_as']  # beam size in arcseconds
hcn = pystruct['INT_VAL_HCN10'] # [K.km/s]
hcn_err = pystruct['INT_UC_HCN10']
co = pystruct['INT_VAL_12CO10'] # [K.km/s]
co_err = pystruct['INT_UC_12CO10']
Sd_sfr = pystruct['INT_VAL_SIGMA_SFR24UM+HA'] * 1e-6  # [Msun/yr/pc2]
Sd_sfr_err = pystruct['INT_UC_SIGMA_SFR24UM+HA'] * 1e-6

# put zeros to nan
mask = hcn == 0
hcn[mask] = np.nan
hcn_err[mask] = np.nan
co[mask] = np.nan
co_err[mask] = np.nan
Sd_sfr[mask] = np.nan
Sd_sfr_err[mask] = np.nan

# compute dense gas surface density
Sd_dense = alpha_hcn * hcn  # [Msun/pc2]
Sd_dense_err = alpha_hcn * hcn_err # [Msun/pc2]

# compute dense gas surface density
Sd_mol = alpha_co * co  # [Msun/pc2]
Sd_mol_err = alpha_co * co_err # [Msun/pc2]

# compute dense gas mass and sfr
# A_pix_pc2 = get_pixel_area(beam_size_as, dist_Mpc, spb=2, pix_shape='hex')
A_beam_pc2 = get_pixel_area(beam_size_as, dist_Mpc, spb=1, pix_shape='gauss')
A_pc2 = A_beam_pc2
m_mol = Sd_mol * A_pc2  # [Msun]
m_mol_err = Sd_mol_err * A_pc2  # [Msun]
m_dense = Sd_dense * A_pc2  # [Msun]
m_dense_err = Sd_dense_err * A_pc2  # [Msun]
sfr = Sd_sfr * A_pc2  # [Msun/yr]
sfr_err = Sd_sfr_err * A_pc2  # [Msun/yr]

# compute total infrared luminosity
tir2sfr = 1.48e-10  # [Msun/yr] Murphy+11
L_tir = sfr / tir2sfr
L_tir_err = sfr_err / tir2sfr

# append to list
det = (hcn / hcn_err) > 3
Refs = Refs + ['Stuber23']*len(sfr[det])
L_HCN = np.concatenate((L_HCN, m_dense[det]/alpha_hcn))
M_dense = np.concatenate((M_dense, m_dense[det]))
L_CO = np.concatenate((L_CO, m_mol[det]/alpha_co))
M_mol = np.concatenate((M_mol, m_mol[det]))
L_TIR = np.concatenate((L_TIR, L_tir[det]))
SFR = np.concatenate((SFR, sfr[det]))
Types = Types + ['parts of galaxies']*len(sfr[det])
Sources = Sources + ['ngc5194']*len(sfr[det])
Refs_long = Refs_long + ['2023A&A...680L..20S']*len(sfr[det])
Dates = Dates + ['12/2023']*len(sfr[det])

# NGC 253 (Beslic+24)

In [17]:
# pystructure file
pystruct_file = wd + 'Data/misc/dense_gas_literature/' + 'ngc0253_data_struct_22as_2022_09_26.npy'
pystruct = np.load(pystruct_file, allow_pickle = True).item()

# get data from structure
dist_Mpc = pystruct['dist_mpc']  # distance to the galaxy in Mpc
incl = pystruct['incl_deg']  # inclination of the galaxy in degree
beam_size_as = pystruct['beam_as']  # beam size in arcseconds
hcn = pystruct['INT_VAL_HCN10'] # [K.km/s]
hcn_err = pystruct['INT_UC_HCN10']
co21 = pystruct['INT_VAL_12CO21'] # [K.km/s]
co21_err = pystruct['INT_UC_12CO21']
Sd_sfr = pystruct['INT_VAL_SIGMA_SFR_FUV_24'] * 1e-6  # [Msun/yr/pc2]

# put zeros to nan
mask = hcn == 0
hcn[mask] = np.nan
hcn_err[mask] = np.nan
co21[mask] = np.nan
co21_err[mask] = np.nan
Sd_sfr[mask] = np.nan

# convert CO(2-1) into CO(1-0) via R_21 calibration from Schinnerer & Leroy 24
R_21 = get_R21_from_SFR(Sd_sfr)
co = co21 / R_21
co_err = co21_err / R_21

# compute dense gas surface density
Sd_dense = alpha_hcn * hcn  # [Msun/pc2]
Sd_dense_err = alpha_hcn * hcn_err # [Msun/pc2]

# compute dense gas surface density
Sd_mol = alpha_co * co  # [Msun/pc2]
Sd_mol_err = alpha_co * co_err # [Msun/pc2]

# compute dense gas mass and sfr
# A_pix_pc2 = get_pixel_area(beam_size_as, dist_Mpc, spb=2, pix_shape='hex')
A_beam_pc2 = get_pixel_area(beam_size_as, dist_Mpc, spb=1, pix_shape='gauss')
A_pc2 = A_beam_pc2
m_mol = Sd_mol * A_pc2  # [Msun]
m_mol_err = Sd_mol_err * A_pc2  # [Msun]
m_dense = Sd_dense * A_pc2  # [Msun]
m_dense_err = Sd_dense_err * A_pc2  # [Msun]
sfr = Sd_sfr * A_pc2  # [Msun/yr]

# compute total infrared luminosity
tir2sfr = 1.48e-10  # [Msun/yr] Murphy+11
L_tir = sfr / tir2sfr

# append to list
det = (hcn / hcn_err) > 3
Refs = Refs + ['Bešlić24']*len(sfr[det])
L_HCN = np.concatenate((L_HCN, m_dense[det]/alpha_hcn))
M_dense = np.concatenate((M_dense, m_dense[det]))
L_CO = np.concatenate((L_CO, m_mol[det]/alpha_co))
M_mol = np.concatenate((M_mol, m_mol[det]))
L_TIR = np.concatenate((L_TIR, L_tir[det]))
SFR = np.concatenate((SFR, sfr[det]))
Types = Types + ['parts of galaxies']*len(sfr[det])
Sources = Sources + ['ngc0253']*len(sfr[det])
Refs_long = Refs_long + ['2024A&A...689A.122B']*len(sfr[det])
Dates = Dates + ['09/2024']*len(sfr[det])

# Save data to table

In [20]:
# make data frame
df = pd.DataFrame({'Name': Refs, 
                   'L_CO [K.km/s.pc2]': L_CO,
                   'L_HCN [K.km/s.pc2]': L_HCN,
                   'L_TIR [Lsun]': L_TIR,
                   'M_mol [Msun]': M_mol,
                   'M_dense [Msun]': M_dense,
                   'SFR [Msun/yr]': SFR,
                   'Type': Types,
                   'Target': Sources,
                   'Reference': Refs_long,
                   'Publication date': Dates,
                  })

# sort table by publication year
refs_yr = df['Publication date']
refs_yr = [float(ref.split('/')[1]) + float(ref.split('/')[0])/12 for ref in refs_yr]  # create year list
df['Ref_yr'] = refs_yr  # add column to table
df = df.sort_values(by=['Ref_yr'])  # sort table by year
df = df.drop(columns=['Ref_yr'])   # delete column


# save to file
database_dir = wd + 'Products/dense_gas_letter/database/'
fname_table = database_dir + 'gao_solomon_literature_compilation_2024.csv'
df.to_csv(fname_table, index=False)

In [19]:
df

,Name,L_CO [K.km/s.pc2],L_HCN [K.km/s.pc2],L_TIR [Lsun],M_mol [Msun],M_dense [Msun],SFR [Msun/yr],Type,Target,Reference,Publication date
294,Chin97,NaN,198.906969,3.366433e+05,NaN,2.983605e+03,0.000050,clouds and clumps,NaN,1997A&A...317..548C,01/1997
293,Chin97,NaN,517.642588,1.383598e+06,NaN,7.764639e+03,0.000205,clouds and clumps,NaN,1997A&A...317..548C,01/1997
292,Chin97,NaN,325.919241,8.629785e+05,NaN,4.888789e+03,0.000128,clouds and clumps,NaN,1997A&A...317..548C,01/1997
291,Chin97,NaN,584.736225,1.067087e+06,NaN,8.771043e+03,0.000158,clouds and clumps,NaN,1997A&A...317..548C,01/1997
295,Chin98,NaN,36.579688,1.330209e+05,NaN,5.486953e+02,0.000020,clouds and clumps,NaN,1998A&A...330..901C,02/1998
...,...,...,...,...,...,...,...,...,...,...,...
1857,Neumann24,1.529159e+07,301652.381839,2.407708e+07,6.651841e+07,4.524786e+06,0.003563,parts of galaxies,ngc4321,2024A&A...691A.121N,11/2024
1856,Neumann24,1.427905e+07,295199.136195,2.727500e+07,6.211388e+07,4.427987e+06,0.004037,parts of galaxies,ngc4321,2024A&A...691A.121N,11/2024
1855,Neumann24,1.285607e+07,251208.273146,3.193398e+07,5.592390e+07,3.768124e+06,0.004726,parts of galaxies,ngc4321,2024A&A...691A.121N,11/2024
1861,Neumann24,1.338569e+07,203119.731364,3.487737e+06,5.822776e+07,3.046796e+06,0.000516,parts of galaxies,ngc4321,2024A&A...691A.121N,11/2024
